<a href="https://colab.research.google.com/github/fergie2/522Lab1/blob/main/ECE590_690.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Quantum Computing Groover Framework

In [ ]:
! pip install pyaztro

Dataset (basic random bigger list) → Partitioning (Master) → Encoding (Q-Worker) → Groover simulation in a jupyter notebook (Q-Worker)

In [ ]:
# ──────────────────────────────────────────────────────────────────────────
# SECTION 1 ▸  Function definitions
# ──────────────────────────────────────────────────────────────────────────
import math
import random
from concurrent.futures import ThreadPoolExecutor, as_completed
from typing import List, Tuple, Any


# Masters node

Dataset:

In [ ]:


def generate_dataset(n=70000):
    """
    Return a list of n integers (0..n-1), shuffled.
    """
    data = list(range(n))
    random.shuffle(data)
    return data

# 1) Generate a dataset of 100 items
dataset = generate_dataset(70000)

print(f"Dataset length: {len(dataset)}")
print("First 10 elements of dataset:", dataset[:10])

Dataset length: 70000
First 10 elements of dataset: [58442, 46352, 1340, 23856, 13389, 46526, 28295, 48990, 38439, 18021]


Partitioning:

classical-master
    ├── slice dataset into ⌈D / N_shard⌉ chunks
    ├── for each chunk:
    │       • serialize to object store / queue
    │       • attach oracle parameters
    │       • pass metadata: n, r_max, ancilla map
    └── gather worker results, post‑process & (optionally) reshard

In [ ]:


def find_optimal_shard_size(num_qubits=64, ancillas=8, r_max=200) -> int:
    """
    Estimate the maximum number of items Grover can handle in one 'search'
    before noise/decoherence becomes too large. Then round up to the
    next power of two for easy circuit indexing.

    Parameters:
      num_qubits: total physical qubits on the quantum worker
      ancillas:   how many qubits you must reserve for your oracle's workspace
      r_max:      the Grover iteration budget you believe is feasible

    Returns:
      shard_size: the recommended items per partition (power of two).
    """
    if ancillas >= num_qubits:
        raise ValueError("Ancilla budget cannot exceed total qubits")

    # 1) Qubit-limited capacity: 2^(n_index) items
    n_index = num_qubits - ancillas
    capacity_qubit = 2 ** n_index

    # 2) Depth-limited capacity: ~ (4 * r_max / pi)^2
    capacity_depth = (4 * r_max / math.pi) ** 2

    # The final capacity is whichever limit is smaller
    capacity = int(min(capacity_qubit, capacity_depth))

    # Round up to the next power of two
    shard_size = 1 << (capacity - 1).bit_length()
    return shard_size


def partition_dataset(
    data: List[Any],
    shard_size: int
) -> List[List[Any]]:
    """
    Split *data* (a list of items) into shards of size *shard_size*.
    The final shard may be smaller or may be padded if that's desired;
    for simplicity here, we won't pad.
    """
    shards = []
    batch = []
    for item in data:
        batch.append(item)
        if len(batch) == shard_size:
            shards.append(batch)
            batch = []
    # Add any leftover
    if batch:
        shards.append(batch)
    return shards

In [ ]:
# 1) Wrap the dataset in (index, value) pairs
indexed_dataset = list(enumerate(dataset))

# 2) Determine an optimal shard size for a 64-qubit worker
optimal_size = find_optimal_shard_size(num_qubits=64, ancillas=8, r_max=200)
print("Optimal shard size:", optimal_size)

# 3) Partition the dataset accordingly
shards = partition_dataset(indexed_dataset, optimal_size)
print(f"Number of shards: {len(shards)}")

# Show the first shard
print("First shard (up to 1-2 lines displayed):", shards[0][:2], "...")

Optimal shard size: 65536
Number of shards: 2
First shard (up to 1-2 lines displayed): [(0, 58442), (1, 46352)] ...


Output: Contribution to workers

In [ ]:
def send_partition_to_worker(partition, target):
    target_value = target
    dataset_worker1 = partition
    print(f"Partition: {dataset_worker1}")
    print(f"Worker received a partition of size {len(partition)}.")
    return dataset_worker1, target_value

In [ ]:
target = 60
print(f"Size Dataset before partitoning: {len(dataset)}")
if len(shards) > 0:
    chosen_partition = shards[0]

    # 2) Send it to your (future) quantum worker
    result = send_partition_to_worker(chosen_partition, target)
    dataset_worker = result[0]
    target_value = result[1]
else:
    print("No partitions to send.")
# print(f"Size Dataset after partitoning: {len(chosen_partition)}")

Size Dataset before partitoning: 70000
Partition: [(0, 58442), (1, 46352), (2, 1340), (3, 23856), (4, 13389), (5, 46526), (6, 28295), (7, 48990), (8, 38439), (9, 18021), (10, 13385), (11, 28663), (12, 5467), (13, 8139), (14, 2364), (15, 62678), (16, 25318), (17, 6650), (18, 12160), (19, 55574), (20, 22489), (21, 20979), (22, 67961), (23, 52257), (24, 30713), (25, 57980), (26, 7699), (27, 21363), (28, 17748), (29, 46034), (30, 46427), (31, 25148), (32, 63250), (33, 62110), (34, 36239), (35, 19920), (36, 1649), (37, 12967), (38, 40583), (39, 1243), (40, 33679), (41, 5565), (42, 32472), (43, 57710), (44, 13908), (45, 48384), (46, 31353), (47, 34772), (48, 13518), (49, 68675), (50, 57498), (51, 184), (52, 24168), (53, 52101), (54, 48759), (55, 64313), (56, 31091), (57, 23690), (58, 10137), (59, 9261), (60, 17712), (61, 42093), (62, 61564), (63, 49997), (64, 49307), (65, 33009), (66, 34472), (67, 9429), (68, 22799), (69, 55121), (70, 464), (71, 16993), (72, 63295), (73, 26201), (74, 43670),

# Workers node

Input from Master

In [ ]:
#output: Dataset: list and Target to find: int

dataset_partition = dataset_worker
print(f"Partition: {target_value}")
print(f"Partition: {dataset_partition}")


Partition: 60
Partition: [(0, 37), (1, 33), (2, 93), (3, 20), (4, 65), (5, 28), (6, 63), (7, 48), (8, 89), (9, 86), (10, 34), (11, 36), (12, 3), (13, 68), (14, 17), (15, 97), (16, 75), (17, 56), (18, 16), (19, 57), (20, 67), (21, 98), (22, 52), (23, 96), (24, 21), (25, 25), (26, 76), (27, 91), (28, 62), (29, 24), (30, 95), (31, 19), (32, 2), (33, 1), (34, 12), (35, 38), (36, 23), (37, 26), (38, 92), (39, 15), (40, 9), (41, 94), (42, 45), (43, 73), (44, 40), (45, 42), (46, 77), (47, 8), (48, 64), (49, 53), (50, 58), (51, 66), (52, 27), (53, 11), (54, 84), (55, 90), (56, 49), (57, 80), (58, 4), (59, 13), (60, 61), (61, 51), (62, 10), (63, 14), (64, 44), (65, 69), (66, 83), (67, 54), (68, 74), (69, 31), (70, 18), (71, 7), (72, 81), (73, 41), (74, 0), (75, 88), (76, 87), (77, 59), (78, 70), (79, 43), (80, 50), (81, 6), (82, 99), (83, 29), (84, 60), (85, 78), (86, 79), (87, 32), (88, 30), (89, 46), (90, 85), (91, 35), (92, 71), (93, 82), (94, 72), (95, 39), (96, 22), (97, 47), (98, 55), (99

## Encoding Part

Basic Encoding

In [ ]:

# Example encoding function (placeholder - replace with your actual encoding logic)
def encode_partition(partition):
    encoded_data = []
    for item in partition:
        # Replace this with your actual encoding logic
        encoded_data.append(bin(item)[2:].zfill(7))  # Example: convert to 7-bit binary
    return encoded_data

# Encode each partition
encoded_partitions = [encode_partition(partition) for partition in dataset_partition]


# Print the encoded partitions
for i, encoded_partition in enumerate(encoded_partitions):
  print(f"Encoded Partition {i+1}: {encoded_partition}")

Encoded Partition 1: ['0000000', '0100101']
Encoded Partition 2: ['0000001', '0100001']
Encoded Partition 3: ['0000010', '1011101']
Encoded Partition 4: ['0000011', '0010100']
Encoded Partition 5: ['0000100', '1000001']
Encoded Partition 6: ['0000101', '0011100']
Encoded Partition 7: ['0000110', '0111111']
Encoded Partition 8: ['0000111', '0110000']
Encoded Partition 9: ['0001000', '1011001']
Encoded Partition 10: ['0001001', '1010110']
Encoded Partition 11: ['0001010', '0100010']
Encoded Partition 12: ['0001011', '0100100']
Encoded Partition 13: ['0001100', '0000011']
Encoded Partition 14: ['0001101', '1000100']
Encoded Partition 15: ['0001110', '0010001']
Encoded Partition 16: ['0001111', '1100001']
Encoded Partition 17: ['0010000', '1001011']
Encoded Partition 18: ['0010001', '0111000']
Encoded Partition 19: ['0010010', '0010000']
Encoded Partition 20: ['0010011', '0111001']
Encoded Partition 21: ['0010100', '1000011']
Encoded Partition 22: ['0010101', '1100010']
Encoded Partition 2

## Grover Simulation

Error-Correction-Ideas:

- Run it multiple time to get a histogram (Error Mitigation):
   - Optimal number of trials: (pi/4)*sqrt(len(index))

- Error correction:
  - using parity bits
  - Corrects qubit flips and phase flips

___

In [ ]:
#just basic implementation (can be deleted!)

# ## Grover Simulation --> needs to be refinded

def grover_simulation(encoded_partition, target):
    """
    Simulates Grover's algorithm on an encoded partition.

    Args:
        encoded_partition: A list of encoded data.
        target: The target value to search for.

    Returns:
        The index of the target value in the encoded_partition if found, otherwise -1
    """

    n = len(encoded_partition)
    iterations = int(math.pi / 4 * math.sqrt(n))  # Optimal number of iterations

    # Placeholder for oracle (replace with your actual oracle implementation)
    def oracle(item):
      return item == bin(target)[2:].zfill(7)

    # Placeholder for diffusion operator (replace with actual quantum implementation)
    def diffusion_operator(amplitudes):
      # Simple average of amplitudes
      avg = sum(amplitudes) / len(amplitudes)
      return [(2 * avg - amp) for amp in amplitudes]

    # Initialization: All amplitudes equal
    amplitudes = [1 / math.sqrt(n)] * n

    for _ in range(iterations):
      # Oracle application: Invert amplitude of marked item.
      for i in range(n):
        if oracle(encoded_partition[i]):
          amplitudes[i] = -amplitudes[i]

      # Diffusion operator
      amplitudes = diffusion_operator(amplitudes)

    # Measurement: Find the index with the highest probability
    max_amplitude_index = amplitudes.index(max(amplitudes, key=abs))

    # Check if the target is actually found based on high amplitude
    if oracle(encoded_partition[max_amplitude_index]):
      return max_amplitude_index
    else:
      return -1


for i, encoded_partition in enumerate(encoded_partitions):
  print(f"Running Grover on partition {i+1} to find {target_value}")

  result_index = grover_simulation(encoded_partition, target_value)
  if result_index != -1:
      print(f"Found {target_value} at index {result_index} in the encoded partition")
      decoded_value = int(encoded_partitions[i][result_index],2) # Decode back to int
      print(f"Decoded value from encoded partition: {decoded_value}")
      original_index_in_partition = dataset_partition[i].index(decoded_value)
      print(f"Value {decoded_value} in original data partition at index: {original_index_in_partition}")
      overall_index = i*len(dataset_partition[i]) + original_index_in_partition
      print(f"Overall index of the target in the complete dataset: {overall_index} and the value at this index: {data[overall_index]}")


      break  # Exit loop after finding the target
  else:
      print(f"Target {target_value} not found in this partition")


Running Grover on partition 1 to find 60
Target 60 not found in this partition
Running Grover on partition 2 to find 60
Target 60 not found in this partition
Running Grover on partition 3 to find 60
Target 60 not found in this partition
Running Grover on partition 4 to find 60
Target 60 not found in this partition
Running Grover on partition 5 to find 60
Target 60 not found in this partition
Running Grover on partition 6 to find 60
Target 60 not found in this partition
Running Grover on partition 7 to find 60
Target 60 not found in this partition
Running Grover on partition 8 to find 60
Target 60 not found in this partition
Running Grover on partition 9 to find 60
Target 60 not found in this partition
Running Grover on partition 10 to find 60
Target 60 not found in this partition
Running Grover on partition 11 to find 60
Target 60 not found in this partition
Running Grover on partition 12 to find 60
Target 60 not found in this partition
Running Grover on partition 13 to find 60
Target 

TypeError: 'int' object is not subscriptable

## decode the result

Interface to the master